In [2]:
from konlpy.tag import Kkma
import os
import re
import numpy as np
import pandas as pd

# 1. 문단 Text 읽어오기

In [3]:
base_dir = 'data/'
text_dirs = os.listdir(base_dir)
text_dirs = sorted(text_dirs)
texts = []
for i in text_dirs[1:]:
    f = open(base_dir+'/'+i)
    data = f.read()
    data = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',data)
    texts += [data]


# 2. 형태소분석
 - 명사만 추출
 - 연속된 명사끼리 n-gram
 - 8글자 이상 단어 제거

In [4]:
kkma = Kkma()

results = []

for text in texts:
    
    tmp = [] 
    result = []
    new_res = []

    for token, pos in kkma.pos(text):

        # 명사가 아닌 단어가 나오면 tmp에 들어간 명사들로 n-gram
        if pos != 'NNG' and len(tmp) != 0:
            for i in range(len(tmp)):
                for j in range(len(tmp)):
                    
                    s = ''.join(tmp[i:j+1])
                    if i > j or len(s)<=1:
                        continue
                    result += [s]
            tmp = []

        # 명사가 아닌 단어 나올 때까지 tmp에 추가
        elif pos == 'NNG':
            tmp += [token]

    # 8 글자 이상 단어 제거 (명사 n-gram에 따라 길어진 경우 제거위해)
    for res in result:
        if len(res) >= 8:
            continue

        new_res += [res]

    results += [ ' '.join(new_res) ]

# 3. Term Weighting
 - TF-IDF
 - TF가 1인 경우는 제거

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_model = TfidfVectorizer(max_df = 25)
tfidf_arr = tfidf_model.fit_transform(results).toarray()

tf_model = CountVectorizer(max_df = 25)
tf_arr = tf_model.fit_transform(results).toarray()

# TF 1 이하는 TF 0으로 간주
for i, c in enumerate(tf_arr):
    tfidf_arr[i][ c <= 1 ] = 0    

# 4. Post Processing: 대표 단어 선택
 - TFIDF가 0이 아닌 TFIDF 기준 Top k 단어들
 - Top k 안에 서로 포함하는 단어가 있으면 가장 긴 것을 선택

In [30]:
final_terms = []
topk = 7 # 조절 가능

for i in range(len(texts)):
    
    new_candidate_terms = []

    # TFIDF가 0이 아닌 단어들 중 TFIDF 기준 Top k 단어들 (k보다 적으면 그만큼만)
    candidate_terms = np.array(tfidf_model.get_feature_names())[(-tfidf_arr[i]).argsort()].tolist()[:min(topk, np.sum(tfidf_arr[i] != 0))]

    # Top k 안에 서로 포함하는 단어가 있으면 가장 긴 것을 선택
    for t_a in candidate_terms:
        
        # 서로 포함하는 단어 리스트
        tmp_list = [ t_b for t_b in candidate_terms if t_b == t_a or t_a in t_b or t_b in t_a ]

        # 길이로 정렬하고 가장 긴 것 선택
        cterm = sorted(tmp_list, key=len)[-1]

        # 이미 포함되지 않은 경우 추가
        if cterm not in new_candidate_terms:
            new_candidate_terms += [ cterm ]

    final_terms += [ new_candidate_terms ]


# 5. 결과 저장

In [31]:
df = pd.DataFrame(final_terms, columns = [ 'Top {}'.format(i+1) for i in range(7) ])
df.fillna("",inplace=True)
df.to_csv("final_terms.csv", index=False)
df

,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7
0,강의,중간,빅데이터분석,부분,,,
1,빅데이터분석,강의,준비,관련,,,
2,정보,단계,지식,지혜,무엇,활용,분석
3,고객이탈,서비스,사람,회사,데이터분석,,
4,데이터마이닝,데이터과학,단어,기법,언급,,
5,카테고리,사람,친구,분류,기저귀,파일링,맥주
6,빅데이터처리,데이터처리기술,,,,,
7,데이터분석,분석적,빅데이터,인사이트,기술,,
8,무엇,빅데이터,데이터분석,,,,
9,데이터분석,인사이트,실행가능,목적,,,
